Подготовим Dataset:

In [20]:
from langchain_community.document_loaders import PyPDFLoader

In [21]:
data_folder = './dataset/'

In [22]:
pdf = PyPDFLoader(
    file_path = data_folder+"Energetic_strategy.pdf",
    extract_images= False
)

In [32]:
pdf.load()[1].page_content

' \n  \nУТВЕРЖДЕНА  \nраспоряжением Правительства  \nРоссийской Федерации  \nот 9 июня 2020 г.  № 1523 -р \n \n \n \n \n  \nЭНЕРГЕТ ИЧЕСКАЯ СТРАТЕГИЯ   \nРоссийской Федерации на период до 2035  года  \n \n \nI. Общие положения  \n \nНастоящая Стратегия разработана в соответствии с Федеральным \nзаконом "О стратегическом планировании в Российской Федерации ", \nПравилами разработки, корректировки, осуществл ения мониторинга и \nконтроля реализации отраслевых документов стратегического \nпланирования Российской Федерации по вопросам, находящимся в \nведении Правительства Российской Федерации, утвержденными \nпостановлением Правительства Российской Федерации от 29  октября 2015  г. \n№ 1162 "Об утверждении Правил разработки, корректировки, \nосуществления мониторинга и контроля реализации отраслевых \nдокументов стратегического планирования Российской Федерации  \nпо вопросам, находящимся в ведении Правительства Российской \nФедера ции", и является межотраслевой стратегией для совокупност

# Разабьём текст и получим векторное хранилище:

Разабьём текс на сигменты:

In [59]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n \n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
    chunk_size=1000, 
    chunk_overlap=128)

fragments = splitter.create_documents([i.page_content for i in pdf.load()])

In [92]:
fragments

[Document(page_content='ПРАВИТЕЛЬСТВО   РОССИЙСКОЙ   ФЕДЕРАЦИИ  \n \nР А С П О Р Я Ж Е Н И Е  \n \nот 9 июня 2020 г.  № 1523 -р   \n \nМОСКВА'),
 Document(page_content='1. Утвердить прилагаемую Энергетическую стратегию Российской \nФедерации на период до 2035 года (далее  - Стратегия).  \n2. Федеральным органам исполн ительной власти руководствоваться \nположениями Стратегии при разработке и корректировке государственных \nпрограмм Российской Федерации и иных документов стратегического \nпланирования.  \n3. Рекомендовать органам государственной власти субъектов \nРоссийской Федера ции руководствоваться положениями Стратегии при \nразработке и корректировке государственных программ субъектов \nРоссийской Федерации и иных документов стратегического планирования.  \n4. Минэнерго России совместно с заинтересованными федеральными \nорганами испол нительной власти:  \nпредставить в 6 -месячный срок в Правительство Российской \nФедерации проект плана мероприятий по реализации Стратегии;  \nоб

Подключим Embeddings из yandex chain:

In [73]:
from yandex_chain import YandexEmbeddings
from langchain.vectorstores import FAISS

In [82]:
embeddings_model = YandexEmbeddings(config="config.json")

In [94]:
vectorstore = FAISS.from_texts(
    [
        i.page_content
        for i in fragments
    ], 
    embedding=embeddings_model
    )
retriever = vectorstore.as_retriever()

In [88]:
out

[[0.04278564453125,
  0.00447845458984375,
  -0.0178985595703125,
  -0.091552734375,
  0.0109100341796875,
  0.055450439453125,
  -0.0224609375,
  -0.14306640625,
  0.001361846923828125,
  -0.085693359375,
  0.06591796875,
  -0.06964111328125,
  0.036041259765625,
  -0.07672119140625,
  -0.063232421875,
  0.0753173828125,
  0.0712890625,
  -0.0404052734375,
  0.00885009765625,
  -0.079345703125,
  0.0802001953125,
  -0.0322265625,
  -0.07470703125,
  -0.0191192626953125,
  0.08807373046875,
  0.032684326171875,
  -0.061248779296875,
  -0.06671142578125,
  0.08251953125,
  0.027923583984375,
  -0.1578369140625,
  0.0256195068359375,
  -0.051177978515625,
  -0.01190185546875,
  -0.0282440185546875,
  -0.070556640625,
  -0.048187255859375,
  -0.0474853515625,
  0.11102294921875,
  0.08453369140625,
  -0.07000732421875,
  0.006500244140625,
  0.03851318359375,
  -0.02435302734375,
  0.014312744140625,
  0.0982666015625,
  -0.0252227783203125,
  0.06427001953125,
  0.0058135986328125,
  0.0

In [80]:
vectorstore = FAISS.from_texts(fragments, embedding=embeddings_model)
retriever = vectorstore.as_retriever()

NameError: name 'retriever' is not defined